Chatbot with TF project

This model is creating a chatbot that is trained on movie dataset. It will understand the text and the split where '?' splits Q with A. For learning part of chatbot it is used RNN model, for prediction it is used LSTM model. Trained on 50000 set with 32 batch and 20 epoch (to see the time performance for TF). 
Chatbot learn from pairs of Q with A from given Q and output with maximum acuracy.

In [2]:
import os
import sys
import requests
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import nltk
from urllib.request import urlopen
from os import path
from zipfile import ZipFile
from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense, Bidirectional
from keras.models import Model, load_model

Using TensorFlow backend.


Get dataset "cornell_movie_dialogs_corpus"

In [3]:
# Load the data

lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [4]:
# Create a dictionary to map each line's id with its text

id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [5]:
# Create a list of all of the conversations' lines' ids

conversations = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversations.append(_line.split(','))

In [6]:
# check for id and conversation sample

for k in conversations[170]:
    print (k, id2line[k])

L989 I don't understand the allure of dehydrated food.  Is this something I should be hip to?
L990 No, Daddy.
L991 So tell me about this dance. Was it fun?
L992 Parts of it.
L993 Which parts?
L994 The part where Bianca beat the hell out of some guy.
L995 Bianca did what?
L996 What's the matter?  Upset that I rubbed off on her?
L997 No -- impressed.


In [7]:
# Sort the sentences into questions (inputs) and answers (targets)

questions = []
answers = []
for conv in conversations:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])
        
# Compare lengths of questions and answers
print(len(questions))
print(len(answers))

221616
221616


In [8]:
# CLEANING TEXT WITH REGEX
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|]", "", text)
#     text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = " ".join(text.split())
    return text

In [9]:
# Clean the dataset

clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

In [10]:
# Find the length of sentences 
lengths = []
lengths.append([len(nltk.word_tokenize(sent)) for sent in clean_questions]) #nltk approach

In [28]:
#lengths = []
# for question in clean_questions:
#     lengths.append(len(question.split()))
# for answer in clean_answers:
#     lengths.append(len(answer.split()))
# # Create a dataframe so that the values can be inspected
# lengths = pd.DataFrame(lengths, columns=['counts'])
# print(np.percentile(lengths, 80))
# print(np.percentile(lengths, 85))
# print(np.percentile(lengths, 90))
# print(np.percentile(lengths, 95))

16.0
19.0
24.0
32.0


In [11]:
print(lengths)

[[25, 17, 11, 17, 15, 2, 29, 2, 28, 11, 8, 11, 51, 10, 12, 2, 7, 8, 11, 3, 21, 5, 29, 13, 2, 4, 4, 17, 3, 4, 9, 1, 21, 7, 3, 15, 11, 4, 4, 23, 1, 1, 3, 4, 6, 2, 10, 2, 1, 32, 6, 20, 29, 9, 12, 3, 13, 37, 3, 2, 5, 4, 12, 5, 21, 12, 14, 31, 5, 19, 4, 14, 11, 11, 6, 13, 29, 18, 4, 24, 11, 2, 6, 2, 8, 4, 12, 12, 4, 1, 35, 7, 13, 3, 10, 9, 25, 19, 9, 3, 5, 3, 16, 12, 5, 21, 16, 7, 14, 8, 3, 6, 18, 6, 9, 8, 7, 4, 15, 2, 15, 8, 3, 22, 11, 13, 5, 6, 22, 14, 1, 17, 5, 5, 6, 8, 9, 14, 9, 2, 3, 6, 6, 3, 9, 4, 11, 3, 11, 26, 7, 4, 8, 9, 12, 10, 8, 22, 4, 9, 7, 25, 10, 26, 15, 11, 30, 7, 5, 16, 15, 5, 6, 6, 3, 4, 7, 7, 4, 25, 11, 17, 13, 7, 19, 7, 7, 6, 7, 1, 7, 16, 5, 4, 5, 16, 23, 5, 4, 2, 5, 1, 6, 13, 11, 3, 8, 17, 12, 26, 13, 12, 10, 4, 13, 5, 2, 6, 3, 5, 6, 3, 16, 2, 4, 6, 5, 25, 27, 7, 7, 8, 4, 12, 4, 5, 22, 13, 4, 14, 4, 53, 10, 36, 10, 8, 5, 18, 4, 6, 5, 17, 7, 18, 9, 3, 18, 15, 18, 18, 17, 5, 2, 9, 8, 10, 11, 6, 16, 17, 7, 12, 13, 7, 24, 6, 5, 17, 10, 7, 3, 10, 20, 7, 4, 20, 10, 2, 5, 8, 1

In [12]:
# Remove questions and answers that are shorter than 1 word and longer than 30 words.
min_line_length = 2
max_line_length = 20

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

for i, question in enumerate(clean_questions):
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

for i, answer in enumerate(short_answers_temp):
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
        
print(len(short_questions))
print(len(short_answers))

138528
138528


In [13]:
#verifying for random Q and A
r = np.random.randint(1,len(short_questions))

for i in range(r, r+3):
    print(short_questions[i])
    print(short_answers[i])
    print()

do not touch me! i am not doing it for you!
then why are you doing it?

then why are you doing it?
because i am a newspaper woman, heaven help me!

and what a way to quit. while you are still champion! that is the way to leave, hildy!
yeah. only only i am not leaving, walter.



In [14]:
#choosing number of samples
INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

num_samples = 50000  # Number of samples to train on.
short_questions = short_questions[:num_samples]
short_answers = short_answers[:num_samples]

#tokenizing the Q and A

short_questions_tok = [nltk.word_tokenize(sent) for sent in short_questions]
short_answers_tok = [nltk.word_tokenize(sent) for sent in short_answers]

In [15]:
#train-validation split
data_size = len(short_questions_tok)

# We will use 80% of selected data for the training
training_input  = short_questions_tok[:round(data_size*(80/100))]
training_input  = [tr_input[::-1] for tr_input in training_input] #reverseing input seq for better performance(backprop)
training_output = short_answers_tok[:round(data_size*(80/100))]

# 20% will use the remaining for validation
validation_input = short_questions_tok[round(data_size*(80/100)):]
validation_input  = [val_input[::-1] for val_input in validation_input] #reverseing input seq for better performance
validation_output = short_answers_tok[round(data_size*(80/100)):]

print('training size', len(training_input))
print('validation size', len(validation_input))

training size 40000
validation size 10000


In [16]:
# For statistics 
# Create a dictionary for the frequency of the vocabulary
vocab = {}
for question in short_questions_tok:
    for word in question:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in short_answers_tok:
    for word in answer:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1            


In [17]:
# Remove rare words from the vocabulary.
# We will aim to replace fewer than 5% of words with <somethink>

threshold = 15
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [18]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 22950
Size of vocab we will use: 2868


In [19]:
#dictionaries to provide a unique integer for each word.
WORD_CODE_START = 1
WORD_CODE_PADDING = 0


word_num  = 2 #number 1 is left for WORD_CODE_START for model decoder later
encoding = {}
decoding = {1: 'START'}
for word, count in vocab.items():
    if count >= threshold: #get vocabularies that appear above threshold count
        encoding[word] = word_num 
        decoding[word_num ] = word
        word_num += 1

print("No. of vocab used:", word_num)

No. of vocab used: 2870


In [20]:
#include unknown token for words not in dictionary
decoding[len(encoding)+2] = '<UNK>'
encoding['<UNK>'] = len(encoding)+2

In [21]:
dict_size = word_num+1
dict_size

2871

In [22]:
#vectorizing dataset for better performance

#:param encoding: encoding dict built by build_word_encoding()
#:param data: list of strings
#:param vector_size: size of each encoded vector

def transform(encoding, data, vector_size=20):   
    
    transformed_data = np.zeros(shape=(len(data), vector_size))
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            try:
                transformed_data[i][j] = encoding[data[i][j]]
            except:
                transformed_data[i][j] = encoding['<UNK>']
    return transformed_data

In [23]:
#encoding training set

encoded_training_input = transform(
    encoding, training_input, vector_size=INPUT_LENGTH)
encoded_training_output = transform(
    encoding, training_output, vector_size=OUTPUT_LENGTH)

print('encoded_training_input', encoded_training_input.shape)
print('encoded_training_output', encoded_training_output.shape)

encoded_training_input (40000, 20)
encoded_training_output (40000, 20)


In [24]:
#encoding validation set
encoded_validation_input = transform(
    encoding, validation_input, vector_size=INPUT_LENGTH)
encoded_validation_output = transform(
    encoding, validation_output, vector_size=OUTPUT_LENGTH)

print('encoded_validation_input', encoded_validation_input.shape)
print('encoded_validation_output', encoded_validation_output.shape)

encoded_validation_input (10000, 20)
encoded_validation_output (10000, 20)


In [25]:
tf.keras.backend.clear_session()
INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

encoder_input = Input(shape=(INPUT_LENGTH,))
decoder_input = Input(shape=(OUTPUT_LENGTH,))

In [26]:
#CREATE THE MODEL
#simple RNN model 
# For the plain Sequence-to-Sequence, will produce the output directly from decoder

from keras.layers import SimpleRNN

encoder = Embedding(dict_size, 128, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder = LSTM(512, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

decoder = Embedding(dict_size, 128, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)
decoder = LSTM(512, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)
# output = TimeDistributed(Dense(output_dict_size, activation="softmax"))(decoder) for deployment

encoder Tensor("lstm_1/transpose_2:0", shape=(None, 20, 512), dtype=float32)
encoder_last Tensor("strided_slice:0", shape=(None, 512), dtype=float32)
decoder Tensor("lstm_2/transpose_2:0", shape=(None, 20, 512), dtype=float32)


In [27]:
from keras.layers import Activation, dot, concatenate

# Equation (7) with 'dot' score from Section 3.1 in the paper.
# Note that we reuse Softmax-activation layer instead of writing tensor calculation
attention = dot([decoder, encoder], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

# Has another weight + tanh layer as described in equation (5) of the paper
output = TimeDistributed(Dense(512, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(dict_size, activation="softmax"))(output)
print('output', output)

attention Tensor("attention/truediv:0", shape=(None, 20, 20), dtype=float32)
context Tensor("dot_2/MatMul:0", shape=(None, 20, 512), dtype=float32)
decoder_combined_context Tensor("concatenate_1/concat:0", shape=(None, 20, 1024), dtype=float32)
output Tensor("time_distributed_2/Reshape_1:0", shape=(None, 20, 2871), dtype=float32)


In [28]:
#Model summary
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 20, 128)      367488      input_2[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 128)      367488      input_1[0][0]                    
____________________________________________________________________________________________

In [29]:
training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = WORD_CODE_START
training_decoder_output = np.eye(dict_size)[encoded_training_output.astype('int')]

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = WORD_CODE_START
validation_decoder_output = np.eye(dict_size)[encoded_validation_output.astype('int')]

In [ ]:
model.fit(x=[training_encoder_input, training_decoder_input], y=[training_decoder_output],
          validation_data=([validation_encoder_input, validation_decoder_input], [validation_decoder_output]),
          #validation_split=0.05,
          batch_size=32, epochs=20)

model.save('model_attention')

/data/jupiter/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 842s 21ms/step - loss: 0.0011 - val_loss: 9.7747e-04
Epoch 2/20
23392/40000 [================>.............] - ETA: 5:16 - loss: 9.4732e-04

TESTING!

In [1]:
def prediction(raw_input):
    clean_input = clean_text(raw_input)
    input_tok = [nltk.word_tokenize(clean_input)]
    input_tok = [input_tok[0][::-1]]  #reverseing input seq
    encoder_input = transform(encoding, input_tok, 20)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
    decoder_input[:,0] = WORD_CODE_START
    for i in range(1, OUTPUT_LENGTH):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return output

def decode(decoding, vector):
#:param decoding: decoding dict built by word encoding
#:param vector: an encoded vector
    text = ''
    for i in vector:
        if i == 0:
            break
        text += ' '
        text += decoding[i]
    return text

In [ ]:
for i in range(20):
    seq_index = np.random.randint(1, len(short_questions))
    output = prediction(short_questions[seq_index])
    print ('Q:', short_questions[seq_index])
    print ('A:', decode(decoding, output[0]))

In [ ]:
raw_input = input()
output = prediction(raw_input)
print (decode(decoding, output[0]))

In [ ]:
FIN!